# Boundary mask builder

### Note: Use the epi-paint kernel for alphashape

What is this used for?
- Once the boundary detection script is run, and the Lamin outline is clearly extracted, this notebook helps to build a polygon around the lamin.
- This polygon is then used to extract the locs within the polygon from all the other channels and then save them for future processing. 

Workflow
- Define the cell folder.
- Search for the file with the boundary data.
- Import the Lamin_boundary.hdf5 file and extract the localizations. 
- Use alphashape to build the polygon around the shape. This should export the polygon. 
- Use the polygon as a mask and then clean all the imaging channels. 

In [ ]:
# Import dependencies

import alphashape as alphashape
from matplotlib.path import Path
import numpy as _np
import os.path as _ospath
import os as _os
import epi_paint_picasso_utilis as eppu


In [ ]:
# Define the folder and the files with the data

folder = '' # <<< Set your folder path here
folder = _ospath.join(folder, 'Cleaned')
file_extn = '.hdf5'
file_names = [f for f in _os.listdir(folder) if f.endswith(file_extn)]

In [ ]:
# Define the output folder

parent_folder, working_folder = _ospath.split(folder)
output_folder = _ospath.join(parent_folder, 'Masked')
if not _ospath.exists(output_folder):
    _os.makedirs(output_folder)

In [ ]:
# Find the file with the string 'boundary' in the file name

for file in file_names:
    if 'boundary' in file:
        boundary_file = _ospath.join(folder, file)
        print('The boundary file {} is found'.format(file))
        file_names.remove(file)
        break

# Load the boundary data
boundary_locs, boundary_info = eppu.load_locs(boundary_file)

# Extract the x and y coordinates of the boundary
boundary_x = boundary_locs['x']
boundary_y = boundary_locs['y']

alpha_points = _np.array([[_x, _y] for _x, _y in zip(boundary_x, boundary_y)])

In [ ]:
# Extract polygon from the boundary data

alpha_shape = alphashape.alphashape(alpha_points, 0.0)
alpha_shape

In [ ]:
# Get the coordinates of the polygon

polygon_coords = _np.array(alpha_shape.exterior.coords)
polygon_path = Path(polygon_coords)
_np.savetxt(_ospath.join(output_folder, 'polygon_coords.csv'), polygon_coords, delimiter=',')

In [ ]:
# Filter the points inside the polygon for the other channels

for file in file_names:
    fpath = _ospath.join(folder, file)
    locs, info = eppu.load_locs(fpath)
    points = _np.column_stack((locs['x'], locs['y']))
    inside_mask = polygon_path.contains_points(points)
    filtered_locs = locs[inside_mask]
    output_path = _ospath.join(output_folder, file)
    eppu.save_locs_withSuffix(output_path, filtered_locs, info, suffix='Masked')
    print('The file {} is processed'.format(file))